In [2]:
import numpy as np
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Data Preparation

In [3]:
from sklearn.preprocessing import LabelEncoder

def encode(train, test):
    le = LabelEncoder().fit(train.species)
    labels = le.transform(train.species)
    classes = list(le.classes_)
    test_ids = test.id
    
    train = train.drop(['species', 'id'], axis=1)  
    test = test.drop(['id'], axis=1)
    
    return train, labels, test, test_ids, classes

train, labels, test, test_ids, classes = encode(train, test)
train.head(1)

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,0.033203,...,0.007812,0.0,0.00293,0.00293,0.035156,0.0,0.0,0.004883,0.0,0.025391


In [4]:
from sklearn.cross_validation import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(labels, n_iter=7, test_size=0.2, random_state=6)

In [5]:
for train_index, test_index in sss:
    X_train, X_test = train.values[train_index], train.values[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

# Classifier

In [6]:
from sklearn.metrics import log_loss
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
test_predictions = lda.predict_proba(X_test)
logl = log_loss(y_test, test_predictions)
print(logl)

0.852995542645


C:\Users\gvenk\Anaconda2\lib\site-packages\sklearn\discriminant_analysis.py:453: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\gvenk\Anaconda2\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


### Performing classification on actual test set

In [7]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
test_predictions = lda.predict_proba(test)

In [8]:
test_predictions5 <- test_predictions^5
test_predictions5

NameError: name 'test_predictions5' is not defined

In [10]:
test_predictions

array([[  5.13544189e-067,   3.40952802e-130,   7.39558872e-123, ...,
          2.87499563e-069,   6.26226255e-085,   7.90201201e-100],
       [  4.69989929e-080,   6.55085288e-153,   1.79700738e-109, ...,
          5.54485806e-040,   1.64195156e-130,   7.99029365e-055],
       [  5.86450917e-044,   2.46794889e-001,   1.52007800e-088, ...,
          1.83444741e-072,   5.84384609e-157,   2.23345117e-018],
       ..., 
       [  1.24685287e-054,   2.24685575e-127,   2.03691783e-112, ...,
          1.33837190e-056,   4.21079923e-114,   7.29252802e-057],
       [  3.12572568e-074,   4.69090887e-133,   7.52191619e-071, ...,
          4.91058637e-032,   6.28179146e-130,   7.85068321e-083],
       [  7.44730125e-097,   4.55628658e-130,   2.43111978e-085, ...,
          1.70794330e-022,   4.20528081e-147,   3.49949212e-056]])

# Submission

In [176]:
submission = pd.DataFrame(test_predictions, columns=classes)

In [177]:
submission["id"] = test_ids
submission.tail()

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
589,1576,1.678022e-32,1.574938e-01,3.052691e-81,3.839102e-29,5.951535e-29,1.081191e-81,2.046718e-121,3.795852e-29,1.574869e-01,...,3.756572e-115,5.097332e-74,1.916082e-133,2.174382e-38,1.956985e-49,1.712025e-04,5.488761e-02,1.154157e-79,7.874750e-163,6.344203e-18
590,1577,5.852780e-19,9.014722e-101,3.282135e-172,2.548416e-13,0.000000e+00,3.760322e-76,1.747866e-88,2.795162e-21,1.666120e-01,...,2.409288e-111,2.067872e-82,4.594498e-61,7.674604e-55,1.666294e-01,1.666294e-01,3.624477e-57,5.677283e-78,4.714038e-136,5.463863e-24
591,1579,1.246853e-54,2.246856e-127,2.036918e-112,6.354985e-20,0.000000e+00,1.994285e-64,4.838702e-108,1.802098e-80,4.848188e-85,...,3.792925e-203,4.313797e-92,3.349110e-121,5.892847e-74,1.011612e-70,3.730513e-62,1.612125e-138,1.338372e-56,4.210799e-114,7.292528e-57
592,1580,3.125726e-74,4.690909e-133,7.521916e-71,8.605904e-22,2.771755e-291,2.939929e-60,7.429960e-53,1.855653e-38,1.111413e-46,...,3.764907e-155,5.846074e-47,3.611489e-149,1.151540e-36,7.658719e-59,4.435737e-27,3.435078e-106,4.910586e-32,6.281791e-130,7.850683e-83
593,1583,7.447301e-97,4.556287e-130,2.431120e-85,1.463458e-27,0.000000e+00,1.415763e-35,8.240679e-85,5.575643e-90,1.203718e-81,...,4.632512e-183,7.447890e-69,1.408725e-163,5.836338e-73,8.453399e-80,4.160309e-50,2.441630e-120,1.707943e-22,4.205281e-147,3.499492e-56


In [178]:
submission.to_csv('submission.csv')